In [7]:
import enum
from typing import List, Dict

class TipoProducto(enum.Enum):
    P10 = "P10"
    P16 = "P16"
    P20 = "P20"
    P30 = "P30"
    P40 = "P40"

class Tolva:
    def __init__(self, material: str, capacidad: float, altura_max: float):
        self.material = material
        self.capacidad = capacidad
        self.altura_max = altura_max
        self.nivel_actual = 0

    def tiempo_vaciado(self, consumo_por_hora: float) -> float:
        toneladas_reales = (self.nivel_actual * self.capacidad) / self.altura_max
        return toneladas_reales / consumo_por_hora if consumo_por_hora > 0 else float('inf')

class Molino:
    def __init__(self, nombre: str, tolvas: Dict[str, Tolva], rendimiento: float):
        self.nombre = nombre
        self.tolvas = tolvas
        self.rendimiento = rendimiento
        self.tipo_producto = None
        self.alimentacion_fresca = 0
        self.ratios = {}

    def set_producto(self, tipo_producto: TipoProducto, alimentacion_fresca: float, ratios: Dict[str, float]):
        self.tipo_producto = tipo_producto
        self.alimentacion_fresca = alimentacion_fresca
        self.ratios = ratios

    def tiempo_vaciado(self, material: str) -> float:
        if material not in self.ratios or material not in self.tolvas:
            return float('inf')
        consumo = (self.alimentacion_fresca * self.ratios[material]) / 100
        return self.tolvas[material].tiempo_vaciado(consumo)

class SistemaAlimentacion:
    def __init__(self):
        self.mc1 = Molino("MC1", {
            "clinker": Tolva("Clinker", 500, 14),
            "puzolana": Tolva("Puzolana", 300, 12),
            "yeso": Tolva("Yeso", 300, 10)
        }, 0.8)
        self.mc2 = Molino("MC2", {
            "clinker": Tolva("Clinker", 300, 9),
            "puzolana_humeda": Tolva("Puzolana Húmeda", 500, 15),
            "puzolana_seca": Tolva("Puzolana Seca", 100, 12),
            "yeso": Tolva("Yeso", 120, 9)
        }, 0.8)
        self.mc3 = Molino("MC3", {
            "clinker": Tolva("Clinker", 60, 100),
            "puzolana": Tolva("Puzolana", 35, 100),
            "yeso": Tolva("Yeso", 30, 100)
        }, 0.5)
        self.silo_blanco = Tolva("Clinker", 500, 10.5)
        self.silo_b = Tolva("Clinker", 500, 10)
        self.galpon = Tolva("Clinker", 1000, 20)
        self.alimentaciones_actuales = set()
        self.carga_banda_transporte = 0

    def set_productos(self):
        for molino in [self.mc1, self.mc2, self.mc3]:
            tipo_producto = self.solicitar_tipo_producto(molino.nombre)
            alimentacion_fresca, ratios = self.obtener_datos_producto(molino.nombre, tipo_producto)
            molino.set_producto(tipo_producto, alimentacion_fresca, ratios)

    def solicitar_tipo_producto(self, nombre_molino: str) -> TipoProducto:
        while True:
            print(f"\nTipos de producto disponibles para {nombre_molino}:")
            for tipo in TipoProducto:
                print(f"- {tipo.value}")
            seleccion = input(f"Seleccione el tipo de producto para {nombre_molino}: ").upper()
            try:
                return TipoProducto(seleccion)
            except ValueError:
                print("Selección inválida. Por favor, intente de nuevo.")

    def obtener_datos_producto(self, nombre_molino: str, tipo_producto: TipoProducto) -> tuple:
        datos_productos = {
            "MC1": {
                TipoProducto.P30: (72, {"clinker": 68.5, "puzolana": 30, "yeso": 1.5}),
                TipoProducto.P40: (64, {"clinker": 58.5, "puzolana": 40, "yeso": 1.5})
            },
            "MC2": {
                TipoProducto.P10: (70, {"clinker": 87, "puzolana_humeda": 10, "yeso": 3}),
                TipoProducto.P16: (80, {"clinker": 81.5, "puzolana_humeda": 16, "yeso": 2.5}),
                TipoProducto.P20: (87, {"clinker": 85.5, "puzolana_humeda": 12, "yeso": 2.5}),
                TipoProducto.P30: (110, {"clinker": 68, "puzolana_humeda": 30, "yeso": 2})
            },
            "MC3": {
                TipoProducto.P30: (37, {"clinker": 67.5, "puzolana": 30, "yeso": 2.5})
            }
        }
        return datos_productos[nombre_molino][tipo_producto]

    def optimizar_alimentacion(self) -> List[str]:
        recomendaciones = []
        recomendaciones.extend(self.optimizar_clinker())
        recomendaciones.extend(self.optimizar_yeso())
        recomendaciones.extend(self.optimizar_puzolana())
        return recomendaciones

    def optimizar_clinker(self) -> List[str]:
        if self.carga_banda_transporte >= 210:
            return ["No se puede alimentar más Clinker. Carga máxima de banda alcanzada."]

        tiempos_vaciado = {
            "MC1": self.mc1.tiempo_vaciado("clinker"),
            "MC2": self.mc2.tiempo_vaciado("clinker"),
            "MC3": self.mc3.tiempo_vaciado("clinker")
        }
        molino_prioritario = min(tiempos_vaciado, key=tiempos_vaciado.get)

        if molino_prioritario == "MC2" and self.mc2.tipo_producto == TipoProducto.P10:
            if self.silo_b.nivel_actual > 0:
                return ["Alimentar Clinker a MC2 desde Silo B"]
            elif self.galpon.nivel_actual > 0:
                return ["Alimentar Clinker a MC2 desde galpón"]
            else:
                return ["No se puede alimentar Clinker para P10 en MC2"]

        if molino_prioritario == "MC3":
            if self.silo_blanco.nivel_actual > 3:
                return ["Alimentar Clinker a MC3 desde Silo Blanco"]
            else:
                return ["Alimentar Clinker a Silo Blanco desde Pretrit"]

        return [f"Alimentar Clinker a {molino_prioritario} desde Pretrit"]

    def optimizar_yeso(self) -> List[str]:
        tiempos_vaciado = {
            "MC1": self.mc1.tiempo_vaciado("yeso"),
            "MC2": self.mc2.tiempo_vaciado("yeso"),
            "MC3": self.mc3.tiempo_vaciado("yeso")
        }
        molino_prioritario = min(tiempos_vaciado, key=tiempos_vaciado.get)

        if molino_prioritario == "MC1":
            if self.mc1.tolvas["yeso"].nivel_actual + 2 <= self.mc1.tolvas["yeso"].altura_max:
                return ["Alimentar Yeso a MC1 por MC1"]
            else:
                return ["Alimentar Yeso a MC1 por MC2"]
        elif molino_prioritario == "MC2":
            return ["Alimentar Yeso a MC2 por MC2"]
        else:  # MC3
            if self.mc1.tolvas["yeso"].nivel_actual > self.mc2.tolvas["yeso"].nivel_actual:
                return ["Alimentar Yeso a MC3 por MC1"]
            else:
                return ["Alimentar Yeso a MC3 por MC2"]

    def optimizar_puzolana(self) -> List[str]:
        recomendaciones = []
        
        tiempo_vaciado_mc1 = self.mc1.tiempo_vaciado("puzolana")
        tiempo_vaciado_mc2 = self.mc2.tiempo_vaciado("puzolana_humeda") if "puzolana_humeda" in self.mc2.tolvas else float('inf')
        
        if tiempo_vaciado_mc1 < tiempo_vaciado_mc2:
            if "Yeso a L1 por L1" not in self.alimentaciones_actuales and "Clinker a L3" not in self.alimentaciones_actuales:
                recomendaciones.append("Alimentar Puzolana Húmeda a MC1 por MC1")
        else:
            if "puzolana_humeda" in self.mc2.tolvas and self.mc2.tolvas["puzolana_humeda"].nivel_actual <= 9:
                if all(x not in self.alimentaciones_actuales for x in ["Yeso a L2 por L2", "Yeso a L3 por L2", "Yeso a L1 por L2", "Puzolana a L1 por L2"]):
                    recomendaciones.append("Alimentar Puzolana Húmeda a 426HO04 por MC2")

        # Puzolana Seca para MC3
        if "Clinker a L3" not in self.alimentaciones_actuales and "Yeso a L3" not in self.alimentaciones_actuales:
            recomendaciones.append("Alimentar Puzolana Seca a MC3 por MC2")

        return recomendaciones

    def ajustar_velocidad_transportador(self, material: str):
        if material == "puzolana":
            return "Ajustar velocidad del transportador a 1100-1150 RPM"
        elif material == "yeso":
            return "Ajustar velocidad del transportador a 850-900 RPM"
        
    def verificar_contaminacion(self):
        return "ADVERTENCIA: Posible contaminación al vaciar el sistema. Proceder con precaución."

def main():
    sistema = SistemaAlimentacion()
    sistema.set_productos()
    
    while True:
        recomendaciones = sistema.optimizar_alimentacion()
        
        print("\nRecomendaciones de alimentación:")
        for rec in recomendaciones:
            print(f"- {rec}")
            if "puzolana" in rec.lower():
                print(f"  {sistema.ajustar_velocidad_transportador('puzolana')}")
            elif "yeso" in rec.lower():
                print(f"  {sistema.ajustar_velocidad_transportador('yeso')}")
            print(f"  {sistema.verificar_contaminacion()}")
            print("  Esperar 3 minutos para vaciado de bandas")
        
        continuar = input("\n¿Desea realizar otra optimización? (s/n): ").lower()
        if continuar != 's':
            break

if __name__ == "__main__":
    main()


Tipos de producto disponibles para MC1:
- P10
- P16
- P20
- P30
- P40



Tipos de producto disponibles para MC2:
- P10
- P16
- P20
- P30
- P40

Tipos de producto disponibles para MC3:
- P10
- P16
- P20
- P30
- P40

Recomendaciones de alimentación:
- Alimentar Clinker a MC1 desde Pretrit
  ADVERTENCIA: Posible contaminación al vaciar el sistema. Proceder con precaución.
  Esperar 3 minutos para vaciado de bandas
- Alimentar Yeso a MC1 por MC1
  Ajustar velocidad del transportador a 850-900 RPM
  ADVERTENCIA: Posible contaminación al vaciar el sistema. Proceder con precaución.
  Esperar 3 minutos para vaciado de bandas
- Alimentar Puzolana Húmeda a 426HO04 por MC2
  Ajustar velocidad del transportador a 1100-1150 RPM
  ADVERTENCIA: Posible contaminación al vaciar el sistema. Proceder con precaución.
  Esperar 3 minutos para vaciado de bandas
- Alimentar Puzolana Seca a MC3 por MC2
  Ajustar velocidad del transportador a 1100-1150 RPM
  ADVERTENCIA: Posible contaminación al vaciar el sistema. Proceder con precaución.
  Esperar 3 minutos para vaciado de bandas